In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('/Users/burak/Developer/MachineLearning/GameBuddy/GameBuddy-Matching/dataset/gamers-info.csv')
# drop result column
df = df.drop('result', axis=1)

In [ ]:
df.head()

In [ ]:
df_games = df['games']
df_games

In [ ]:
#encode games by using one hot encoding
df_games_encoded = df_games.str.get_dummies(sep=',')
# remove if there is a [ or ] in the game name
df_games_encoded = df_games_encoded.rename(columns=lambda x: x.replace('[', '').replace(']', ''))
df_games_encoded

In [ ]:
df_keywords_encoded = df['keywords'].str.get_dummies(sep=',')
df_keywords_encoded = df_keywords_encoded.rename(columns=lambda x: x.replace('[', '').replace(']', ''))
df_keywords_encoded

In [ ]:
# combine df_games_encoded and df_keywords_encoded to df
df = df.drop(['games', 'keywords'], axis=1)
print(df.shape)
df = df.join(df_games_encoded)
print(df.shape)
df = df.join(df_keywords_encoded)
print(df.shape)

In [ ]:
df.columns

In [ ]:
df.shape

In [ ]:
df.sample(5)

In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
df['country'] = le.fit_transform(df['country'])

In [ ]:
df.shape

Scaling The Data

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

# scale all the columns except
# country and result
df_scaled = df.copy()
df_scaled = df_scaled.drop(['country', 'age'], axis=1)
df_scaled = pd.DataFrame(scaler.fit_transform(df_scaled), columns=df_scaled.columns)

df_scaled.sample(5)


In [ ]:
df.shape

Principal Component Analysis

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
pca = PCA()

pca_df = pca.fit_transform(df_scaled)
# fit PCA on the scaled data
pca.fit(df_scaled)

# find the accuracy of pca
pca.explained_variance_ratio_

In [ ]:
df.shape

In [ ]:
pca_df = pca.fit_transform(df_scaled)

In [ ]:
df.shape

In [ ]:
total_explained_variance = pca.explained_variance_ratio_.cumsum()
n_over_95 = len(total_explained_variance[total_explained_variance >= 0.65])
n_to_reach_95 = df_scaled.shape[1] - n_over_95

print(total_explained_variance.size)
print(n_over_95)
print(n_to_reach_95)

In [ ]:
df.shape

In [ ]:
print(f"Number features: {n_to_reach_95}\nTotal Variance Explained: {total_explained_variance[n_over_95]}")

In [ ]:
pca = PCA(n_components=n_over_95)
df_pca = pca.fit_transform(df_scaled)

print(pca.explained_variance_ratio_.cumsum())

In [ ]:
df.shape

Clustering

In [ ]:
from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.metrics import silhouette_score, davies_bouldin_score

In [ ]:
n_clusters = 20
cluster_cnt = [i for i in range(2, n_clusters, 2)]

scores = []
db_scores = []

for n in cluster_cnt:
    hac = AgglomerativeClustering(n_clusters=n)
    hac.fit(df_pca)

    score = silhouette_score(df_pca, hac.labels_)
    db_scores.append(davies_bouldin_score(df_pca, hac.labels_))
    scores.append(silhouette_score(df_pca, hac.labels_))



In [ ]:
print("Size of cluster_cnt: ", len(cluster_cnt))
print("Size of scores: ", len(scores))
print("Size of db_scores: ", len(db_scores))
print("Hac lables: ", hac.labels_.size)
print("Size of df", df.shape)

Cluster Evaluation

In [ ]:
#df = pd.DataFrame(columns=['Cluster Score'], index=[i for i in range(2, len(db_scores)+2)])
#add a new column at the begging of df named cluster score and assign the hac labels to it
print("Before merge: ", df.shape)
df.insert(0, 'Cluster Score', hac.labels_)
print("After merge: ", df.shape)

print('Max Value:\nCluster #', df[df['Cluster Score'] == df['Cluster Score'].max()])
print('\nMin Value:\nCluster #', df[df['Cluster Score'] == df['Cluster Score'].min()])

In [ ]:
df.shape

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import random

In [ ]:
import pickle

In [ ]:
with(open('matching.pkl', 'wb')) as f:
    pickle.dump(df, f)

with(open('games-encoded.pkl', 'wb')) as f:
    pickle.dump(df_games_encoded, f)